In [ ]:
#imports tools and constants definitions
import numpy as np
from astropy.io import ascii 
import matplotlib.pyplot as plt 
from matplotlib import cm
%matplotlib inline

In [ ]:
#reads the data and tests for correct input
data = ascii.read('Astr511HW1data.dat.txt',format='tab') #commented_head use this in jupyter notebooks
print(data[:10])
 ##    ra        dec   run     Ar      u      g      r      i      z  uErr  gErr  rErr  iErr  zErr    pmL    pmB    pmErr

In [16]:
#computes absolute magnitude [Ivezić et al. 2008]

In [4]:
#computes metallicity [Bond et al. 2010]